In [155]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np

In [156]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [157]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [158]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [159]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data)
market_data_scaled[:5, :]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [160]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(market_data_scaled)

# Copy the crypto names from the original data
column_names = list(df_market_data.columns)
column_names_dict = {}
for i in range(len(column_names)):
    column_names_dict[i] = column_names[i]
df_market_data_scaled = df_market_data_scaled.rename(columns=column_names_dict)





In [161]:
# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index(df_market_data.index)

print(df_market_data_scaled.describe())
# Display sample data

df_market_data

       price_change_percentage_24h  price_change_percentage_7d  \
count                    41.000000                4.100000e+01   
mean                      0.000000                1.895503e-16   
std                       1.012423                1.012423e+00   
min                      -4.981042               -1.682027e+00   
25%                      -0.127467               -7.066688e-01   
50%                       0.077497               -1.906843e-01   
75%                       0.331280                4.931931e-01   
max                       1.919812                2.572251e+00   

       price_change_percentage_14d  price_change_percentage_30d  \
count                 4.100000e+01                 4.100000e+01   
mean                  2.707861e-17                 2.978647e-17   
std                   1.012423e+00                 1.012423e+00   
min                  -2.217108e+00                -1.393153e+00   
25%                  -6.299628e-01                -4.605580e-01   
50%

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


---

### Find the Best Value for k Using the Original Data.

In [162]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [163]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for n in k:
    k_model = KMeans(n_clusters=n, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

inertia


[287.0,
 212.1233420748626,
 165.13675233446807,
 79.02243535120977,
 66.41305075939437,
 61.63408811068221,
 46.07446744103021,
 43.39269411734827,
 32.91322265971525,
 30.46911129327035,
 26.480101374398934]

In [164]:
# Create a dictionary with the data to plot the Elbow curve
elbow_inertia = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_inertia_df = pd.DataFrame(elbow_inertia)
elbow_inertia_df

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,165.136752
3,4,79.022435
4,5,66.413051
5,6,61.634088
6,7,46.074467
7,8,43.392694
8,9,32.913223
9,10,30.469111


In [165]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_1 = elbow_inertia_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve Scaled K-Means", 
    xticks=k
)
elbow_1


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** There is a clear elbow *kink* at k=4. I would use k=4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [166]:
# Initialise the K-Means model using the best value for k
model_k4 = KMeans(n_clusters=4, random_state=1)
model_k4

KMeans(n_clusters=4, random_state=1)

In [167]:
# Fit the K-Means model using the scaled data
model_k4.fit(df_market_data_scaled)


KMeans(n_clusters=4, random_state=1)

In [168]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predict_k4 = model_k4.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
predict_k4

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [169]:
# Create a copy of the DataFrame
df_market_data_scaled_predicted = df_market_data.copy()

In [170]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_predicted["predicted k4"] = predict_k4

# Display sample data
df_market_data_scaled_predicted.head(10)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted k4
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,2
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,2
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,2
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195,2
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186,2
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756,2
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408,2


In [171]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter_1 = df_market_data_scaled_predicted.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted k4',
    hover_cols='coin_id',
    title="Predicted Clusters Using K=4"
)

scatter_1

:NdOverlay   [predicted k4]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimise Clusters with Principal Component Analysis.

In [172]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [173]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_scaled_pca = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
market_data_scaled_pca[:5,:]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [174]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

In [175]:
# sum of the variance
round(np.sum(pca.explained_variance_ratio_), 3)

0.895

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.895 of variance within the dataset is explained by the 3 components of the PCA model. 

In [176]:
# Create a new DataFrame with the PCA data.
df_market_data_scaled_pca = pd.DataFrame(market_data_scaled_pca, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
df_market_data_scaled_pca["coinid"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled_pca = df_market_data_scaled_pca.set_index('coinid')

# Display sample data
(df_market_data_scaled_pca.head(5))

,PC1,PC2,PC3
coinid,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [177]:
# Create a list with the number of k-values from 1 to 11
k # from above

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [178]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for n in k:
    k_model = KMeans(n_clusters=n, random_state=1)
    k_model.fit(df_market_data_scaled_pca)
    inertia.append(k_model.inertia_)

inertia

[256.8740855678924,
 182.33953007775642,
 135.44240762454208,
 49.665496651797355,
 38.672582175917945,
 34.50757653758152,
 23.72836478127475,
 18.574523440183185,
 15.572672210433332,
 11.40766657209691,
 8.058821888712728]

In [179]:
elbow_inertia_pca = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_inertia_pca_df= pd.DataFrame(elbow_inertia_pca)
elbow_inertia_pca_df

,k,inertia
0,1,256.874086
1,2,182.339530
2,3,135.442408
3,4,49.665497
4,5,38.672582
5,6,34.507577
6,7,23.728365
7,8,18.574523
8,9,15.572672
9,10,11.407667


In [180]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_2 = elbow_inertia_pca_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve PCA 3 dimensions", 
    xticks=k
)
elbow_2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** At k=4 there is a clear elbow. k=4 would be the appropriate k-value


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** This is the same k value as the original data. however the inertia value has reduced from 80 to 50.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [181]:
# Initialise the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4, random_state=1)
model_pca

KMeans(n_clusters=4, random_state=1)

In [182]:
# Fit the K-Means model using the PCA data
model_pca.fit(df_market_data_scaled_pca)

KMeans(n_clusters=4, random_state=1)

In [183]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predict_pca = model_pca.predict(df_market_data_scaled_pca)
# Print the resulting array of cluster values.
predict_pca


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [184]:
# Create a copy of the DataFrame with the PCA data
df_market_data_pca_predicted = df_market_data_scaled_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_pca_predicted['k4 pca predict'] = predict_pca

# Display sample data
df_market_data_pca_predicted.head(5)

,PC1,PC2,PC3,k4 pca predict
coinid,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [185]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter2 = df_market_data_pca_predicted.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='k4 pca predict',
    hover_cols='coin_id',
    title="Predicted Clusters Using PCA K=4"
)

scatter2

:NdOverlay   [k4 pca predict]
   :Scatter   [PC1]   (PC2)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [186]:
# Composite plot to contrast the Elbow curves
elbow_plot = (elbow_1 + elbow_2).cols(2)
elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [187]:
# Composite plot to contrast the clusters
scatter = (scatter_1 + scatter2).cols(2)
scatter

:Layout
   .NdOverlay.I  :NdOverlay   [predicted k4]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [k4 pca predict]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The effect of dimensionality reduction using PCA and using the same clustering algorithm (K-Means) provides a tighter cluster (less dispersion or variance) and increased cluster separation. This is illustrated quantitatively by using the Calinski-Harabasz score on k=4 as demonstrated below. The PCA dimensionality reduction has a higher score which indicates better cluster separation and reduced cluster dispersion.

In [188]:
# Use the Calinski-Harabasz score to as a measure of cluster fit

model_init = KMeans(n_clusters=4, random_state=1)
labels_init = model_init.fit(df_market_data_scaled).labels_
CHS_k4_init = metrics.calinski_harabasz_score(df_market_data_scaled.to_numpy(), labels_init)

model_pca = KMeans(n_clusters=4, random_state=1)
labels_pca = model_init.fit(df_market_data_scaled_pca).labels_
CHS_k4_pca = metrics.calinski_harabasz_score(df_market_data_scaled_pca.to_numpy(), labels_pca)

results_CHS_df = pd.DataFrame({
    'Methods': ['Scaled K-Means k=4', 'Scaled K-Means PCA k=4'],
    "Score": [CHS_k4_init, CHS_k4_pca]
})
results_CHS_df.hvplot.bar(
    x='Methods',
    y='Score',
    title="Calinski-Harabasz Score Comparison",
)

:Bars   [Methods]   (Score)